In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

%matplotlib inline

In [2]:
import sys
import os
cd=os.getcwd()
print(cd)
data=os.getcwd()+r'/Actuary Data/train_set.csv'

C:\Users\joogl\InsuranceClaimPredictions


In [3]:
print(sys.version)

3.6.3 |Anaconda custom (64-bit)| (default, Oct 15 2017, 03:27:45) [MSC v.1900 64 bit (AMD64)]


In [4]:
# Load test data
test_data = os.getcwd() + r'/Actuary Data/test_set.csv'
df_test = pd.read_csv(test_data)

In [5]:
# load data
df = pd.read_csv(data, nrows=1000000)
df.head(10)

C:\Users\joogl\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Row_ID,Household_ID,Vehicle,Calendar_Year,Model_Year,Blind_Make,Blind_Model,Blind_Submodel,Cat1,Cat2,...,Var5,Var6,Var7,Var8,NVCat,NVVar1,NVVar2,NVVar3,NVVar4,Claim_Amount
0,1,1,3,2005,2005,K,K.78,K.78.2,D,C,...,1.008912,0.261040,0.907793,-0.077998,M,-0.231530,-0.266117,-0.272337,-0.251419,0.0
1,2,2,2,2005,2003,Q,Q.22,Q.22.3,B,C,...,1.240851,0.432987,-0.726459,0.204785,O,-0.231530,-0.266117,-0.272337,-0.251419,0.0
2,3,3,1,2005,1998,AR,AR.41,AR.41.1,B,?,...,-0.971487,-1.405797,-0.837048,-1.176858,F,-0.231530,-0.266117,-0.272337,-0.251419,0.0
3,4,3,1,2006,1998,AR,AR.41,AR.41.1,B,?,...,-0.971487,-1.405797,-0.837048,-1.176858,F,-0.231530,-0.266117,-0.272337,-0.251419,0.0
4,5,3,2,2005,2001,D,D.20,D.20.0,J,C,...,0.812656,2.112691,1.534462,2.347260,F,-0.231530,-0.266117,-0.272337,-0.251419,0.0
5,6,3,2,2006,2001,D,D.20,D.20.0,J,C,...,0.812656,2.112691,1.534462,2.347260,F,-0.231530,-0.266117,-0.272337,-0.251419,0.0
6,7,4,1,2006,2001,AJ,AJ.129,AJ.129.3,G,C,...,0.580718,0.551128,0.416289,-0.024395,M,-0.231530,-0.266117,-0.272337,-0.251419,0.0
7,8,4,2,2006,2002,AQ,AQ.17,AQ.17.1,B,C,...,0.527193,-0.023200,-0.701884,0.226664,M,-0.231530,-0.266117,-0.272337,-0.251419,0.0
8,9,4,3,2005,2002,AQ,AQ.17,AQ.17.1,B,C,...,0.527193,-0.023200,-0.701884,0.226664,M,-0.231530,-0.266117,-0.272337,-0.251419,0.0
9,10,5,1,2005,1995,BW,BW.3,BW.3.0,D,?,...,0.176312,0.283264,0.969232,-0.792339,N,2.054683,-0.266117,-0.272337,-0.251419,0.0


In [6]:
print(df.columns)
print(df_test.columns)

Index(['Row_ID', 'Household_ID', 'Vehicle', 'Calendar_Year', 'Model_Year',
       'Blind_Make', 'Blind_Model', 'Blind_Submodel', 'Cat1', 'Cat2', 'Cat3',
       'Cat4', 'Cat5', 'Cat6', 'Cat7', 'Cat8', 'Cat9', 'Cat10', 'Cat11',
       'Cat12', 'OrdCat', 'Var1', 'Var2', 'Var3', 'Var4', 'Var5', 'Var6',
       'Var7', 'Var8', 'NVCat', 'NVVar1', 'NVVar2', 'NVVar3', 'NVVar4',
       'Claim_Amount'],
      dtype='object')
Index(['Row_ID', 'Household_ID', 'Vehicle', 'Calendar_Year', 'Model_Year',
       'Blind_Make', 'Blind_Model', 'Blind_Submodel', 'Cat1', 'Cat2', 'Cat3',
       'Cat4', 'Cat5', 'Cat6', 'Cat7', 'Cat8', 'Cat9', 'Cat10', 'Cat11',
       'Cat12', 'OrdCat', 'Var1', 'Var2', 'Var3', 'Var4', 'Var5', 'Var6',
       'Var7', 'Var8', 'NVCat', 'NVVar1', 'NVVar2', 'NVVar3', 'NVVar4'],
      dtype='object')


In [7]:
# dummy code claims
df['claim'] = df.Claim_Amount.apply(lambda x: 0 if x == 0 else 1)

In [8]:
# separate feature matrix from labels (claims in {0 = No claim, 1 = Claim})
# only continuous variables are included in the model
X = df[['Var5', 'Var6', 'Var7', 'Var8', 'NVVar1', 'NVVar2', 'NVVar3', 'NVVar4']].values
y = df['claim'].values

In [9]:
print(X)

[[ 1.0089121  0.2610397  0.9077934 ..., -0.2661168 -0.2723372 -0.2514189]
 [ 1.2408507  0.4329871 -0.7264591 ..., -0.2661168 -0.2723372 -0.2514189]
 [-0.9714867 -1.4057974 -0.8370476 ..., -0.2661168 -0.2723372 -0.2514189]
 ..., 
 [-0.8525439 -1.4151551 -1.0090742 ..., -0.2661168 -0.2723372 -0.2514189]
 [-0.8525439 -1.4151551 -1.0090742 ..., -0.2661168 -0.2723372 -0.2514189]
 [-0.8822796 -1.4771497 -1.2548265 ..., -0.2661168 -0.2723372 -0.2514189]]


In [10]:
# split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 42)

In [11]:
print(X_train)

[[ 0.6164006  0.5511279  0.4162889 ..., -0.2661168 -0.2723372 -0.2514189]
 [-1.149901  -1.3613484 -0.9722114 ..., -0.2661168 -0.2723372 -0.2514189]
 [ 0.3547263  0.3616348  0.9938067 ..., -0.2661168  5.1057515 -0.2514189]
 ..., 
 [ 0.1287348 -0.3647552 -0.8984857 ..., -0.2661168 -0.2723372 -0.2514189]
 [ 0.1644177  0.0072126 -0.8124724 ..., -0.2661168 -0.2723372 -0.2514189]
 [ 2.1745522  1.8085663  1.7064883 ..., -0.2661168 -0.2723372 -0.2514189]]


In [12]:
# scale features to have same variance
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [13]:
# fit logistic regression model
model = LogisticRegression(random_state=42)
model.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [14]:
# get in and out of sample accuracy
y_pred = model.predict(X_train)
in_acc = accuracy_score(y_pred, y_train)
y_pred = model.predict(X_test)
out_acc = accuracy_score(y_pred, y_test)
print('In-sample accuracy: {}\nOut-of-sample accuracy: {}'.format(in_acc, out_acc))

In-sample accuracy: 0.99194
Out-of-sample accuracy: 0.99176


In [15]:
print("==============================================================================")
print("==============================================================================")

In [16]:
"""
# (1) we now have model of whether someone filed a claim or not
# (2) of those who did (claim=1), we need model of y = claim amount

Full Model:
a) given X, use (1) to assess if claim will be filed
b) if for (1), y = 1, then estimate claim amount using (2)
"""

'\n# (1) we now have model of whether someone filed a claim or not\n# (2) of those who did (claim=1), we need model of y = claim amount\n\nFull Model:\na) given X, use (1) to assess if claim will be filed\nb) if for (1), y = 1, then estimate claim amount using (2)\n'

In [17]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

# get data for train set where claim=1
df_claims = df[df['claim']==1]

X_claims = df_claims[['Var5', 'Var6', 'Var7', 'Var8', 'NVVar1', 'NVVar2', 'NVVar3', 'NVVar4']].values
y_claims = df_claims['Claim_Amount'].values

"""
# Random forests provide free cross-validation
kf = KFold(n_splits=2)
kf.get_n_splits(X_claims)

print(kf)  

for train_index, test_index in kf.split(X_claims):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_claims_train, X_claims_test = X_claims[train_index], X_claims[test_index]
    y_claims_train, y_claims_test = y_claims[train_index], y_claims[test_index]
"""

X_claims_train, X_claims_test, y_claims_train, y_claims_test = \
                    train_test_split(X_claims, y_claims, test_size = 0.25, random_state = 42)
    
sc = StandardScaler()
X_claims_train = sc.fit_transform(X_claims_train)
X_claims_test = sc.transform(X_claims_test)

In [18]:
#------------------------#
# MODEL 1: Random Forest #
#------------------------#

regr = RandomForestRegressor(max_depth=8, random_state=42)

scores = cross_val_score(regr, X_claims_train, y_claims_train, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

regr.fit(X_claims_train, y_claims_train)

print(regr.feature_importances_)

#-------------------------#
y_claims_pred = regr.predict(X_claims_train)
# need accuracy metric
y_claims_pred = regr.predict(X_claims_test)
# need accuracy metric
# print('In-sample accuracy: {}\nOut-of-sample accuracy: {}'.format(in_acc, out_acc))
#-------------------------#

print(y_claims_pred)
print(y_claims_test)

print(mean_squared_error(y_claims_test, y_claims_pred))

Accuracy: -0.05 (+/- 0.05)
[ 0.13676609  0.20632711  0.16041709  0.33397543  0.04484945  0.05467583
  0.0466511   0.0163379 ]
[ 192.1696557   206.69372887  143.74793985 ...,  107.4150727   143.74793985
  149.89417106]
[  12.6273     335.4804       3.510095  ...,  381.7336      30.9326
    0.6957895]
140595.267086


In [19]:
from sklearn.ensemble import ExtraTreesRegressor

clf_extratrees = ExtraTreesRegressor(n_estimators=8, max_depth=None,
     min_samples_split=2, random_state=0)
scores_extratrees = cross_val_score(clf_extratrees, X_claims_train, y_claims_train, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_extratrees.mean(), scores_extratrees.std() * 2))

Accuracy: -0.46 (+/- 0.23)


In [20]:
from sklearn.datasets import load_iris
from sklearn.ensemble import AdaBoostRegressor

iris = load_iris()
clf_adaboostr = AdaBoostRegressor(n_estimators=100)
scores_adaboostr = cross_val_score(clf_adaboostr, X_claims_train, y_claims_train, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_adaboostr.mean(), scores_adaboostr.std() * 2))

Accuracy: -0.49 (+/- 0.57)


In [21]:
from sklearn.ensemble import GradientBoostingRegressor

est = GradientBoostingRegressor(n_estimators=8, learning_rate=.5,
     max_depth=8, random_state=0).fit(X_claims_train, y_claims_train)

scores_gradboost = cross_val_score(est, X_claims_train, y_claims_train, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_gradboost.mean(), scores_gradboost.std() * 2))

mean_squared_error(y_claims_test, est.predict(X_claims_test))

Accuracy: -0.23 (+/- 0.09)


156627.00885784245

In [22]:
from sklearn.neighbors import KNeighborsRegressor

neigh = KNeighborsRegressor(n_neighbors=2)    # 2 is the best number of K
scores_neigh = cross_val_score(neigh, X_claims_train, y_claims_train, cv=50)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_neigh.mean(), scores_gradboost.std() * 2))
# neigh.fit(X, y)

Accuracy: -0.78 (+/- 0.09)
